In [1]:
# train_detector_fast_fixed.py
# pip install transformers datasets torch scikit-learn accelerate

import numpy as np
import torch
import sklearn.metrics as metrics
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

# -------------------------------
# ⚙️ CONFIGURATION
# -------------------------------
MODEL_NAME = "distilbert-base-uncased"   # Lightweight BERT
NUM_LABELS = 2                           # 0 = Non-suicidal, 1 = Suicidal
FAST_MODE = True                         # Quick test mode
TEST_SIZE = 0.2                          # 80% train, 20% test

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🧠 Using device: {device}")

# -------------------------------
# 1️⃣ LOAD DATASET
# -------------------------------
print("\n🔹 Loading dataset from Hugging Face...")
ds = load_dataset("cypsiSAS/transformed_Suicidal_ideation")
print("✅ Original splits:", list(ds.keys()))

# Some Hugging Face datasets only have a "train" split
if "train" in ds and len(ds.keys()) == 1:
    print("⚠️ Dataset has no test/validation split — creating one manually...")
    ds = ds["train"].train_test_split(test_size=TEST_SIZE, seed=42)
    ds = DatasetDict({
        "train": ds["train"],
        "test": ds["test"]
    })
print("✅ Dataset splits after fix:", list(ds.keys()))

# Detect text column dynamically
sample = ds["train"][0]
text_col = "formatted" if "formatted" in sample else ("text" if "text" in sample else "content")
print(f"🧾 Using text column: '{text_col}'")

# -------------------------------
# 2️⃣ HANDLE LABELS
# -------------------------------
if "labels" not in ds["train"].features and "label" not in ds["train"].features:
    print("⚠️ No labels found — creating synthetic ones from keywords...")

    def create_labels(examples):
        suicidal_keywords = [
            "kill myself", "end my life", "commit suicide", "want to die",
            "suicidal", "end it all", "take my own life", "better off dead",
            "not worth living", "want to disappear", "hurt myself", "self harm"
        ]
        labels = []
        for text in examples[text_col]:
            text_lower = text.lower()
            labels.append(1 if any(k in text_lower for k in suicidal_keywords) else 0)
        return {"labels": labels}

    ds = ds.map(create_labels, batched=True)
else:
    label_col = "label" if "label" in ds["train"].features else "labels"
    if label_col != "labels":
        ds = ds.rename_column(label_col, "labels")

# Label distribution
if "labels" in ds["train"].features:
    y = np.array(ds["train"]["labels"])
    print(f"\n📊 Label distribution — Non-suicidal (0): {np.sum(y==0)}, Suicidal (1): {np.sum(y==1)}")

# -------------------------------
# 3️⃣ FAST MODE (subset for quick testing)
# -------------------------------
if FAST_MODE:
    print("\n⚡ Fast Mode: using a small subset for quick training.")
    ds["train"] = ds["train"].select(range(min(50, len(ds["train"]))))
    ds["test"] = ds["test"].select(range(min(10, len(ds["test"]))))

# -------------------------------
# 4️⃣ TOKENIZATION
# -------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess(examples):
    return tokenizer(
        examples[text_col],
        truncation=True,
        padding="max_length",
        max_length=256
    )

print("\n🔹 Tokenizing...")
ds = ds.map(preprocess, batched=True)
print("✅ Tokenization complete.")

format_cols = ["input_ids", "attention_mask"]
if "labels" in ds["train"].features:
    format_cols.append("labels")
ds.set_format(type="torch", columns=format_cols)

# -------------------------------
# 5️⃣ MODEL INITIALIZATION
# -------------------------------
print("\n🔹 Loading model...")
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)
model.to(device)
print("✅ Model loaded successfully.")

# -------------------------------
# 6️⃣ METRIC FUNCTION
# -------------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision = metrics.precision_score(labels, preds, zero_division=0)
    recall = metrics.recall_score(labels, preds, zero_division=0)
    f1 = metrics.f1_score(labels, preds, zero_division=0)
    acc = metrics.accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


/Users/chinmayabharadwajhs/day/final/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🧠 Using device: cpu

🔹 Loading dataset from Hugging Face...
✅ Original splits: ['train']
⚠️ Dataset has no test/validation split — creating one manually...
✅ Dataset splits after fix: ['train', 'test']
🧾 Using text column: 'formatted'
⚠️ No labels found — creating synthetic ones from keywords...


Map: 100%|██████████| 13233/13233 [00:00<00:00, 156182.13 examples/s]



📊 Label distribution — Non-suicidal (0): 963, Suicidal (1): 51965

⚡ Fast Mode: using a small subset for quick training.

🔹 Tokenizing...


Map: 100%|██████████| 10/10 [00:00<00:00, 1982.09 examples/s]

✅ Tokenization complete.

🔹 Loading model...



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded successfully.


In [2]:
# -------------------------------
# 7️⃣ TRAINING ARGUMENTS
# -------------------------------
training_args = TrainingArguments(
    output_dir="out_model",
    do_eval=True,
    eval_steps=10,
    save_steps=20,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=5,
    fp16=torch.cuda.is_available(),
)


In [3]:
# -------------------------------
# 8️⃣ TRAINER SETUP
# -------------------------------
eval_split = "validation" if "validation" in ds else "test"

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds[eval_split],
    compute_metrics=compute_metrics,
)


In [4]:
# -------------------------------
# 9️⃣ TRAIN & EVALUATE
# -------------------------------
print("\n🚀 Starting training...")
trainer.train()

print("\n📈 Evaluating model on unseen test data...")
results = trainer.evaluate(ds[eval_split])

print("\n✅ Evaluation results:")
print(f"Accuracy : {results['eval_accuracy']*100:.2f}%")
print(f"Precision: {results['eval_precision']*100:.2f}%")
print(f"Recall   : {results['eval_recall']*100:.2f}%")
print(f"F1 Score : {results['eval_f1']*100:.2f}%")



🚀 Starting training...


/Users/chinmayabharadwajhs/day/final/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
5,0.585400
10,0.383300
15,0.200100
20,0.206400
25,0.072100
30,0.051100
35,0.186900


/Users/chinmayabharadwajhs/day/final/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)



📈 Evaluating model on unseen test data...


/Users/chinmayabharadwajhs/day/final/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)



✅ Evaluation results:
Accuracy : 90.00%
Precision: 90.00%
Recall   : 100.00%
F1 Score : 94.74%
